# Zastosowania DFT
## Laboratorium 9 - Metody Obliczeniowe w Nauce i Technice

In [2]:
import numpy as np
import scipy.linalg as salg
import matplotlib.pyplot as plt
import time
from functools import reduce

## Zadanie 1. Prosty ranking wierzchołków.

## Zadanie 2.